# Bibliotecas

In [161]:
import pandas as pd

# Limpeza dos dados

## Lendo os arquivos

In [162]:
df_americanas = pd.read_csv(f"Dados/produtos_americanas.csv", dtype = {'ean': str})
df_amazon = pd.read_csv(f"Dados/produtos_amazon.csv", dtype = {'ean': str})

In [163]:
df_amazon["ean"].value_counts()

0713477350430    57
9095247871972    51
7908414420304     2
7319625661149     1
7465214678507     1
                 ..
0708570152586     1
8263430934112     1
6973553520094     1
3041412361256     1
6934177770012     1
Name: ean, Length: 1017, dtype: int64

## Retirando títulos com palavras indesejadas

In [164]:
bad_words = ["acessório", "acessorio", "adaptador", "alargadores", "anel", "arduino", "atomiser", "avatar",
             "aparelho de som", "automotivo", "axiom",
             "bandeja", "biometria", "bike", "boroscópio", "box", 
             "cable", "cabo", "caixa", "campainha", "calculator", "calculadora", "capa", "capinha", "card",
             "case", "caso", "carregador", "carro"
             "chassi", "chaveiro", "chave seletora", "clip",
             "coletor", "collar", "conjunto", "comoda", "cômoda", "cordão", "cortador", "cosplay",
             "drone", "eon", "eixo", "eixos", "espelho", "filme", "franquia", "gancho", "grânulos", "gaveta",
             "handmade", "hemobblo", "hemobllo", "hd externo", "jbl", "junta", "kit",
             "lente", "lixa", "lurrose",
             "m-disc", "mágico", "magico", "mesa", "microsd", "monitor", "montagem", "moochy",
             "netcna", "óculos", "oculos", "ornament",
             "papel", "parede", "part", "partes", "pedal", "pen drive", "pendrive", "pneu", "pneus", "power bank",
             "prato", "prettyzoom", "pretyzoom", "proteção", "protetor", "pulseira",
             "raspberry", "radiador", "rank de memória", "realidade virtual", "refrigerador", "regeneradora",
             "relogio", "relógio", "remoção", "reparo", "reposição", "reposiçao", "resfriador", "retrovisor", "roda",
             "ronyme", "rv", "scanner", "sensor", "smart tv", "soquete", "stick", "streaming box", "suporte",
             "tablet", "tampa", "tomada", "tomadas", "tradutor", "turbopower",
             "ultra-sensível", "upgrade", "upgrades",
             "vador", "veado", "vedação", "verbatim", "vidro", "vr", "xilofone", "xylophone",
             "watch", "zawa", "zbox"
            ]

print(f"Amazon: {df_amazon.shape}\nAmericanas: {df_americanas.shape}")

df_amazon['titulo_cb'] = df_amazon['titulo'].str.lower()
df_americanas['titulo_cb'] = df_americanas['titulo'].str.lower()

df_amazon_ft = df_amazon[~df_amazon["titulo_cb"].str.contains('|'.join(bad_words))]
df_americanas_ft = df_americanas[~df_americanas["titulo_cb"].str.contains('|'.join(bad_words))]
print(f"\nFiltrado:\nAmazon: {df_amazon_ft.shape}\nAmericanas: {df_americanas_ft.shape}")

Amazon: (4333, 4)
Americanas: (1032, 4)

Filtrado:
Amazon: (1911, 5)
Americanas: (876, 5)


In [ ]:
#df_amazon_ft['url'].loc[970]

In [ ]:
#df_amazon_ft[350:].head(50)

## Encontrando EANs repetidos na mesma loja

Removendo produtos com o mesmo título

In [ ]:
df_amazon_ft = df_amazon_ft.loc[~df_amazon_ft["titulo"].duplicated(keep = 'first')]
df_americanas_ft = df_americanas_ft.loc[~df_americanas_ft["titulo"].duplicated(keep = 'first')]

In [ ]:
print(f"Amazon: {df_amazon_ft.shape}\nAmericanas: {df_americanas_ft.shape}")

Amazon: (1738, 5)
Americanas: (815, 5)


Verificando Títulos de EANs duplicados

In [ ]:
ama_vc_ean = df_amazon_ft[df_amazon_ft["ean"].notnull()]["ean"].value_counts()
ama_duplicados = ama_vc_ean[ama_vc_ean > 1].index[0]
df_amazon_ft[df_amazon_ft['ean'] == ama_duplicados]

,url,titulo,ean,descricao,titulo_cb
262,https://www.amazon.com.br/Smartphone-Multilase...,Smartphone Multilaser G Max 2 2GB + 64GB Tela ...,7908414420304,Sobre este item Câmera Tripla 8MP + VGA + V...,smartphone multilaser g max 2 2gb + 64gb tela ...
2632,https://www.amazon.com.br/Smartphone-Multilase...,Smartphone Multilaser G Max 2 Dourado 64GB 2GB...,7908414420304,Sobre este item Câmera Tripla 8MP + VGA + V...,smartphone multilaser g max 2 dourado 64gb 2gb...


In [ ]:
ame_vc_ean = df_americanas_ft[df_americanas_ft["ean"].notnull()]["ean"].value_counts()
ame_duplicados = ame_vc_ean[ame_vc_ean > 1].index
df_americanas_ft[df_americanas_ft["ean"].isin(ame_duplicados)].sort_values(by = ['ean']).head(1)

,descricao,ean,titulo,url,titulo_cb
459,Características gerais Redmi Modelo - Note 10 ...,6934177734519,Redmi Note 10 Pro Dual Sim 128 gb 6 Gb Azul,https://www.americanas.com.br/produto/4431267188,redmi note 10 pro dual sim 128 gb 6 gb azul


In [ ]:
df_americanas_ft['loja'] = "americanas"
df_amazon_ft['loja'] = "amazon"

In [ ]:
frames = [df_americanas_ft, df_amazon_ft]
df_final = pd.concat(frames, ignore_index = True)

In [ ]:
vc_ean = df_final[df_final["ean"].notnull()]["ean"].value_counts()
duplicados = vc_ean[vc_ean > 1].index
df_final_dup = df_final[df_final["ean"].isin(duplicados)].sort_values(by = ['ean'])

## Fazendo match

In [ ]:
colunas = ["descricao_1", "ean_1", "titulo_1", "url_1",	"titulo_cb_1", "loja_1",
           "descricao_2", "ean_2", "titulo_2", "url_2",	"titulo_cb_2", "loja_2"]

df_pares = pd.DataFrame(columns = colunas)
df_pares

,descricao_1,ean_1,titulo_1,url_1,titulo_cb_1,loja_1,descricao_2,ean_2,titulo_2,url_2,titulo_cb_2,loja_2


In [ ]:
len(df_asdf)

139

In [ ]:
len(df_final_dup)

39

In [ ]:
lista_ean = []

for i in range(len(df_final_dup)):
    
    if df_final_dup.iloc[i]['ean'] not in lista_ean:
        
        lista_ean.append(df_final_dup.iloc[i]['ean'])
        
        indice = df_final_dup.index[df_final_dup['ean'] == df_final_dup.iloc[i]['ean']].tolist()
        
        for j in range(len(indice)-1):
            
            for k in range(j + 1, len(indice)):
            
                df_pares = df_pares.append({'descricao_1' : df_final_dup.loc[indice[j]]['descricao'] , 'ean_1' : df_final_dup.loc[indice[j]]['ean'],
                                        'titulo_1' : df_final_dup.loc[indice[j]]['titulo'], 'url_1' : df_final_dup.loc[indice[j]]['url'],
                                        'titulo_cb_1' : df_final_dup.loc[indice[j]]['titulo_cb'], 'loja_1' : df_final_dup.loc[indice[j]]['loja'],
                                        
                                        'descricao_2' : df_final_dup.loc[indice[k]]['descricao'] , 'ean_2' : df_final_dup.loc[indice[k]]['ean'],
                                        'titulo_2' : df_final_dup.loc[indice[k]]['titulo'], 'url_2' : df_final_dup.loc[indice[k]]['url'] ,
                                        'titulo_cb_2' : df_final_dup.loc[indice[k]]['titulo_cb'], 'loja_2' : df_final_dup.loc[indice[k]]['loja']}
                                        , ignore_index=True)

In [ ]:
#df_pares[df_pares['ean_1']=="6934177738180"][['descricao_1', 'ean_1', 'titulo_1', 'descricao_2', 'ean_2', 'titulo_2', 'loja_1', 'loja_2', 'url_1', 'url_2']]
df_pares[['descricao_1', 'ean_1', 'titulo_1', 'descricao_2', 'ean_2', 'titulo_2', 'loja_1', 'loja_2', 'url_1', 'url_2']]

,descricao_1,ean_1,titulo_1,descricao_2,ean_2,titulo_2,loja_1,loja_2,url_1,url_2
0,Características gerais Redmi Modelo - Note 10 ...,6934177734519,Redmi Note 10 Pro Dual Sim 128 gb 6 Gb Azul,Características gerais Redmi Modelo - Note 10 ...,6934177734519,Redmi Note 10 Pro Dual Sim 128 Gb Cinza 6 Gb,americanas,americanas,https://www.americanas.com.br/produto/4431267188,https://www.americanas.com.br/produto/4431295470
1,Desempenho superior com Snapdragon 860O que ve...,6934177738180,Poco X3 Pro Dual sim 256GB 8GB ram - Preto,Sistema operacional Nome do sistema operaciona...,6934177738180,Pocophone Poco X3 Pro Dual sim 256GB Preto 8GB...,americanas,americanas,https://www.americanas.com.br/produto/4540396296,https://www.americanas.com.br/produto/4511090415
2,Modelo - Note 11Ano - 2022Número da peça - Not...,6934177767173,Redmi Note 11 Dual Sim 128gb 6gb Ram Câmera 50...,>4 ram 128 gb >Tela--------- 6.43 polegadas fh...,6934177767173,Smartphone Redmi Note 11 4 ram 128 gb grafite,americanas,americanas,https://www.americanas.com.br/produto/4986718691,https://www.americanas.com.br/produto/4946490047
3,O Redmi 10 Prime traz a quad câmera dos sonhos...,6941059643364,Smartphone Redmi 10 Prime Dual Sim 64 Gb Carbo...,"Smartphone Xiaomi Redmi Note 9, Midnight Grey,...",6941059643364,"Smartphone Xiaomi Redmi Note 9, Midnight Grey,...",americanas,americanas,https://www.americanas.com.br/produto/4636373591,https://www.americanas.com.br/produto/4720006460
4,"Enviamos para todo o Brasil, com o melhor praz...",6941399050617,Celular Gamer Realme dual chip 64gb Preto C21Y,Fotografia profissional no seu bolso Descubra ...,6941399050617,Realme C21Y RMX3261 Dual 64 gb - Preto,americanas,americanas,https://www.americanas.com.br/produto/4612063731,https://www.americanas.com.br/produto/4472405410
5,O Poco M3 Pro 5G é um smartphone Android de bo...,6954176809902,Smartphone Poco M3 Pro 5G Dual sim 128 gb cool...,O Poco X3 gt é um smartphone Android de bom ní...,6954176809902,Smartphone Poco X3 gt 5G 128gb 8gb Ram Azul,americanas,americanas,https://www.americanas.com.br/produto/4827284371,https://www.americanas.com.br/produto/4616816171
6,O Poco M3 Pro 5G é um smartphone Android de bo...,6954176809902,Smartphone Poco M3 Pro 5G Dual sim 128 gb cool...,Redmi 9T 128GB 4GB Ram Dual Gray Versão Global...,6954176809902,Redmi 9T 128GB 4GB Ram Dual Gray Versão Global,americanas,americanas,https://www.americanas.com.br/produto/4827284371,https://www.americanas.com.br/produto/4409136552
7,O Poco X3 gt é um smartphone Android de bom ní...,6954176809902,Smartphone Poco X3 gt 5G 128gb 8gb Ram Azul,Redmi 9T 128GB 4GB Ram Dual Gray Versão Global...,6954176809902,Redmi 9T 128GB 4GB Ram Dual Gray Versão Global,americanas,americanas,https://www.americanas.com.br/produto/4616816171,https://www.americanas.com.br/produto/4409136552
8,Este Aparelho Possui a Tecnologia nfc O Smartp...,6973553520636,Smartphone umidigi bison Dual Sim lte 6.3 6GB/...,O Smartphone bison X10 da umidigi é ultra-resi...,6973553520636,Smartphone umidigi bison X10 Dual Sim lte 6.53...,americanas,americanas,https://www.americanas.com.br/produto/4848602560,https://www.americanas.com.br/produto/4707302977
9,O redmí Note 8 2021 é um smartphone Android co...,7892509113274,Smartphone Note 8 64gb Tela 6.3 Preto mod. 2021,O Samsung Galaxy A01 é um dos lançamentos da m...,7892509113274,Smartphone Samsung Galaxy A01 32GB 4G Wi-Fi Te...,americanas,americanas,https://www.americanas.com.br/produto/4432145203,https://www.americanas.com.br/produto/4450820196


## Salvando Pares

In [ ]:
df_pares.to_csv(f"Dados/pares_smartphones.csv", index = False)